# Percolation in Erdős-Rényi networks

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

rng = np.random.default_rng()

As you should know by now, Erdős-Rényi (ER) networks are random networks where $N$ nodes are randomly connected such that the probability that a pair of nodes is connected is $p$. In this exercise, we will analyze the percolation properties of ER graphs as a function of their average degree $\langle k\rangle$, which, as you know, depends on $p$.

## a)Visualization

We start by visualizing the percolation process in a relatively small ER network. Create ER networks, each with $N = 500$ nodes and average degree $\langle k \rangle = [0.7, 0.9, 1.1, 1.3, 1.5]$, respectively. Visualize each network with the nodes in the largest component in red and the rest of the nodes in grey. Then answer the MyCourses quiz question.

In [ ]:
def link_probability_of_ER_network(n, avg_degree):
    '''
    Return the probability of a link between any two nodes in an ER network 
    given number of nodes n and average degree avg_degree.
    '''
    p = 0 # Replace!
    # TODO: calculate the link probability p of the ER model for the given n and avg_degree
    # YOUR CODE HERE
    # YOUR CODE HERE

    p = avg_degree / (n - 1)

    return p

def largest_component(G):
    '''
    Return the largest component of a graph G as a set of nodes.
    '''
    nodes_in_largest_component = set() # Replace!
    # TODO: Find the largest component of G as a set of nodes
    # HINT: Use nx.connected_components(G) to get a list of components
    # YOUR CODE HERE

    largest_component = max(nx.connected_components(G), key=len)
    nodes_in_largest_component = set(largest_component)
    
    return nodes_in_largest_component

In [ ]:
# NO NEED TO MODIFY THIS FUNCTION
def draw_largest_component(ax, G):
    '''
    Draw a networkx graph G on a matplotlib axis ax, with the largest component 
    highlighted in red and the rest of the nodes in grey.
    '''
    nodes_in_lc = largest_component(G)
    k_spring = 2 * G.number_of_edges() * (G.number_of_nodes())**(-1.5)
    pos = nx.spring_layout(G, k=k_spring)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=nodes_in_lc, 
                           node_size=10, node_color="red", ax=ax, alpha=0.5)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=set(G.nodes()) - set(nodes_in_lc), 
                           node_size=10, node_color="grey", ax=ax, alpha=0.5)
    nx.draw_networkx_edges(G, pos=pos, width=0.5, alpha=0.3, ax=ax)

In [ ]:
fig_vis, axes_vis = plt.subplots(1, 5, figsize=(20, 3.5))
n = 500
avg_degrees = [0.7, 0.9, 1.1, 1.3, 1.5]
for i, avg_degree in enumerate(avg_degrees):
    p = link_probability_of_ER_network(n, avg_degree)
    G = nx.fast_gnp_random_graph(n, p, seed=rng)
    draw_largest_component(axes_vis[i], G)
    axes_vis[i].set_title(r"$\langle k \rangle = {:.1f}, p = {:.2g}$".format(avg_degree, p))

In [ ]:
# Save the figure in the current directory
fig_vis.savefig("percolation_in_ER_visualization.pdf", bbox_inches="tight")

## b) Largest component size
To understand how the size of the largest component depends on the average degree, and how they differ for different network sizes, make a plot of the fraction of nodes in the largest component as a function of the average degree $\langle k \rangle$ scanned between $0$ and $2.5$ in steps of $0.1$, i.e., $\langle k \rangle = [0, 0.1, 0.2, \dots, 2.5]$. Consider networks with $N = [50, 500, 5000, 50000]$ nodes. By looking at the plots, estimate the percolation threshold, i.e., the value of $\langle k \rangle$ for which a giant component starts to appear. Enter this value into MyCourses.

In [ ]:
# Plot the fraction of the largest component as a function of average degree <k>
network_sizes = [50, 500, 5000, 50000]
avg_degrees = np.arange(0, 2.6, 0.1) # average degree <k>

fig_diffsizes, axes_diffsizes = plt.subplots(1, 4, figsize=(16, 3))
for i, n in enumerate(network_sizes):
    # fraction of nodes in the largest component
    largest_component_fractions = [] # Replace!

    # TODO: Generate a ER network with for each average degree in avg_degrees,  
    # calculate the fraction of nodes in the largest component 
    # (i.e., the largest component size normalized by the total number of nodes) 
    # and append it to largest_component_fractions
    # YOUR CODE HERE

    for avg_degree in avg_degrees:
        p = link_probability_of_ER_network(n, avg_degree)
        G = nx.fast_gnp_random_graph(n, p, seed=rng)
        largest_component_size = len(largest_component(G))
        largest_component_fraction = largest_component_size / n
        largest_component_fractions.append(largest_component_fraction)
    
    axes_diffsizes[i].plot(avg_degrees, largest_component_fractions, 'o', color='black')
    axes_diffsizes[i].set_title(r"$N = {}$".format(n))
    axes_diffsizes[i].set_xlabel(r"$\langle k \rangle$")
    axes_diffsizes[i].set_ylim(0, 1)
axes_diffsizes[0].set_ylabel("fraction of largest component")

In [ ]:
# Save the figure in the current directory
fig_diffsizes.savefig("percolation_in_ER_different_sizes.pdf", bbox_inches="tight")

## c) Percolation threshold and susceptibility
Another, more elegant way to find out when the percolation transition happens is to find the point at which the probability for the largest component size growth is the highest, as the control parameter is changed very little (here $\langle k \rangle$ or $p$). Consider a situation where $\langle k \rangle$ is changed so slightly that a single link is added between the largest component and a randomly selected node that is not in the largest component. The expected change in the largest component size in this situation is called the *susceptibility*, and it should have a large value at the percolation threshold. The susceptibility depends on the size distribution of all  components other than the largest, and it can be calculated with the following formula:
\begin{equation}
\chi = \frac{(\sum_i i^2 C(i)) - S_{max}^2}{(\sum_i i C(i)) - S_{max}} \,,
\end{equation}
where $C(i)$ is the number of components with $i$ nodes. Calculate the susceptibility $\chi$ for networks with $N = 100000$ and $\langle k \rangle = [0, 0.1, 0.2, \dots, 2.5]$, and plot $\chi$ as a function of $\langle k \rangle$. Then answer the MyCourses quiz. 

In [ ]:
# Plot the fraction of the second largest component as a function of average degree <k>
n = 100000
avg_degrees = np.arange(0, 2.6, 0.1)

fig_susc, axes_susc = plt.subplots(2, 1, figsize=(6, 6))
largest_component_fractions = [] # Replace!
susceptibilities = [] # Replace!

# TODO: Generate a ER network with for each average degree in avg_degrees, and
# 1. calculate the fraction of nodes in the largest component and append it to 
#    largest_component_fractions,
# 2. calculate the susceptibility and append it to susceptibilities.

# YOUR CODE HERE

for avg_degree in avg_degrees:
    p = link_probability_of_ER_network(n, avg_degree)
    G = nx.fast_gnp_random_graph(n, p, seed=rng)
    largest_component_size = len(largest_component(G))
    largest_component_fraction = largest_component_size / n
    largest_component_fractions.append(largest_component_fraction)

    # Calculate the susceptibility
    numerator = 0
    denominator = 0

    # \chi = \frac{(\sum_i i^2 C(i)) - S_{max}^2}{(\sum_i i C(i)) - S_{max}} \,,
    # where C(i) is the number of components of size i
    # and S_{max} is the size of the largest component

    component_sizes = [len(component) for component in nx.connected_components(G)]
    component_sizes = np.bincount(component_sizes)

    for i, component_size in enumerate(component_sizes):
        numerator += i**2 * component_size
        denominator += i * component_size

    numerator -= largest_component_size**2
    denominator -= largest_component_size

    susceptibility = numerator / denominator
    susceptibilities.append(susceptibility)

axes_susc[0].plot(avg_degrees, largest_component_fractions, 'o-', color='red')
axes_susc[1].plot(avg_degrees, susceptibilities, 'o-', color='black')
axes_susc[0].set_title(r"$N = {}$".format(n))
axes_susc[1].set_xlabel(r"$\langle k \rangle$")
for i in range(2):
    axes_susc[i].set_xlim(0, 2.5)
axes_susc[0].set_ylim(top=1)
axes_susc[0].set_ylabel("fraction of largest component")
axes_susc[1].set_ylabel("susceptibility")
axes_susc[1].grid()

In [ ]:
# Save the figure in the current directory
fig_susc.savefig("percolation_in_ER_susceptibility.pdf", bbox_inches="tight")